<a href="https://colab.research.google.com/github/returaj/cs6910/blob/assginment1_akash/question_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install wandb

In [3]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
from google.colab import drive
drive.mount('/content/drive')
!cp drive/MyDrive/DLASSIGNMENT/activation_func.py .
import activation_func
!cp drive/MyDrive/DLASSIGNMENT/loss_func.py .
import loss_func
!cp drive/MyDrive/DLASSIGNMENT/runner.py .
!cp drive/MyDrive/DLASSIGNMENT/feed_forward_nn.py .
!cp drive/MyDrive/DLASSIGNMENT/measure.py .
!cp drive/MyDrive/DLASSIGNMENT/optimizer.py .

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import numpy as np
from keras.datasets import fashion_mnist

import wandb

from feed_forward_nn import FNN
from measure import accuracy
from optimizer import SGD, MomentumGD, NesterovGD, Rmsprop, Adam, Nadam
from activation_func import Sigmoid, Relu, Tanh
from loss_func import CrossEntropy, MeanSquaredError
from sklearn.model_selection import train_test_split
#from runner import Runner

In [6]:
np.random.seed(2)
class Runner1(object):
  def __init__(self, isFashionMnistData=True):
    if isFashionMnistData:
      self.initialize_fashion_mnist_data()
    else:
      self.initialize_data()

  def initialize_fashion_mnist_data(self):
    (X, self.y), (X_test, self.y_test) = fashion_mnist.load_data()
    self.X = np.array([x.flatten() for x in X]) / 255
    self.X_test = np.array([x.flatten() for x in X_test]) / 255
    

  def initialize_data(self):
    raise NotImplementedError("Please implement this method if you need other dataset.")

  @staticmethod
  def get_loss_function(key):
    mapper = {
      "cross_entropy": CrossEntropy,
      "mse": MeanSquaredError
    }
    assert key in mapper
    return mapper[key]

  @staticmethod
  def get_activation_function(key):
    mapper = {
      "sigmoid": Sigmoid,
      "relu": Relu,
      "tanh": Tanh
    }
    assert key in mapper
    return mapper[key]

  @staticmethod
  def get_optimizer(key):
    mapper = {
      "sgd": SGD,
      "momentum_gd": MomentumGD,
      "nesterov_gd": NesterovGD,
      "rmsprop": Rmsprop,
      "adam": Adam,
      "nadam": Nadam
    }
    assert key in mapper
    return mapper[key]

  @staticmethod
  def train(X_train, y_train,params,do_val, wandb_log, print_after=500):
    """
    X_train: (batch_size(B), data_size(N))
    y_train: (batch_size(B))
    params: dict(
      batch_size: int,
      epochs: int,
      alpha: float,
      optimizer: One of (SGD, MomentumGD, NesterovGD, Rmsprop, Adam, Nadam),
      hidden_layers_size: list(layer_size),
      act_func: One of (Sigmoid, Relu, Tanh),
      reg: float,
      init: One of (random, xavier),
      loss_func: One of (CrossEntropy, MeanSquaredError)
    )
    """
    if do_val:
      X_train, X_val, y_train, y_val = train_test_split(
          X_train, y_train, test_size=0.1, random_state=10)

    data_size, input_size = X_train.shape
    output_size = 10
    batch_size, epochs = params['batch_size1'], params['epochs1']
    act_func, loss_func = params['act_func1'](), params['loss_func1']()
    model = FNN(
      input_size         = input_size,
      output_size        = output_size,
      hidden_layers_size = params['hidden_layers_size1'],
      act_func           = act_func,
      reg                = params['reg1'],
      init               = params['init1'],
      loss_func          = loss_func,
    )
    optimizer = params['optimizer1'](model, params['alpha1'])
    for ep in range(1, epochs+1):
      ids = np.arange(data_size)
      np.random.shuffle(ids)
      start, end = 0, batch_size
      while end > start:
        x, y = X_train[ids[start:end]], y_train[ids[start:end]]
        optimizer.optimize(x, y)
        start, end = end, min(data_size, end+batch_size)
      # log
      train_loss = loss_func.error(X_train, y_train, model)
      estimate_y_train = Runner1.predict(X_train, model)
      train_acc = accuracy(estimate_y_train, y_train)
      val_loss, val_acc = "NotDefined", "NotDefined"
      if do_val:
        val_loss = loss_func.error(X_val, y_val, model)
        estimate_y_val = Runner1.predict(X_val, model)
        val_acc = accuracy(estimate_y_val, y_val)

      Runner1.logger(train_loss,train_acc,val_loss,val_acc,ep,wandb_log)
    #estimate_y_test = Runner.predict(X_test,model)
    #test_acc = accuracy(estimate_y_test,Y_test)
    #wandb.log({"test_accuracy":test_acc})
    return model
  @staticmethod
  def test(model,X_test,Y_test):
    estimate_y_test = Runner1.predict(X_test,model)
    test_acc = accuracy(estimate_y_test,Y_test)
    wandb.log({"test_accuracy_mse":test_acc})
    return model


  def logger(train_loss, train_acc, val_loss, val_acc,step,wandb_log):
    # print(f"TrainingLoss: {train_loss}, TrainingAccuracy: {train_acc}")
    # print(f"ValidationLoss: {val_loss}, ValidationAccuracy: {val_acc}")
    if wandb_log:
      wandb.log({
        "epochs_mse": step,
        "Training_Loss_mse": train_loss,
        "Training_Accuracy_mse": train_acc,
        "Validation_Loss_mse": val_loss,
        "Validation_Accuracy_mse": val_acc,  
      }
      )




  @staticmethod
  def predict(X, model):
    """
    X: (batch_size(B), data_size(N))
    return np.array of size: (batch_size(B),)
    """
    prob = model.forward(X)[-1]
    return np.argmax(prob, axis=1)


def run_wandb():
  wandb.init(project="CS6910_ASSIGNMENT", entity="cs21m003_cs21d406")
  config = wandb.config
  loss_name = "ce" if config.loss_func1 == "cross_entropy" else "mse"
  wandb.run.name=f"e_{config.epochs1}_bs_{config.batch_size1}_hl_{config.hidden_layers1}_hn_{config.hidden_nodes1}_init_{config.weight_init1}_ac_{config.act_func1}_ls_{loss_name}_opt_{config.optimizer1}_rate_{config.learning_rate1}_reg_{config.reg1}"
  hidden_layers_size = [config.hidden_nodes1] * config.hidden_layers1
  runner =Runner1()
  params = {
    "batch_size1"        : config.batch_size1,
    "epochs1"            : config.epochs1,
    "alpha1"             : config.learning_rate1,
    "optimizer1"         : runner.get_optimizer(config.optimizer1),
    "hidden_layers_size1": hidden_layers_size,
    "act_func1"          : runner.get_activation_function(config.act_func1),
    "reg1"               : config.reg1,
    "init1"              : config.weight_init1,
    "loss_func1"         : runner.get_loss_function(config.loss_func1),
  }
  model=runner.train(runner.X, runner.y, params, do_val=True, wandb_log=True)
  runner.test(model,runner.X_test,runner.y_test)

def do_hyperparameter_search_using_wandb():
  sweep_config = {
    "name": "Bayesian Sweep",
    "method": "bayes",
    "metric":{
      "name": "ValidationAccuracy",
      "goal": "maximize"
    },
    "parameters":{
      "epochs1": {"values": [5, 10]}, 
      "batch_size1": {"values": [16, 32, 64]}, 
      "hidden_layers1": {"values": [3, 4, 5]}, 
      "hidden_nodes1": {"values": [32, 64, 128]},
      "reg1": {"values": [0, 0.0005, 0.5]},
      "weight_init1": {"values": ['random', 'xavier']} , 
      "act_func1": {"values": ["sigmoid", "tanh", "relu"]}, 
      "loss_func1": {"values": ["mse"]}, 
      "learning_rate1": {"values": [1e-3, 1e-4]},   
      "optimizer1": {"values": ["sgd", "momentum_gd", "nesterov_gd", "rmsprop", "adam", "nadam"]}, 
    }
  }
  
  sweep_id = wandb.sweep(sweep_config, project = "CS6910_ASSIGNMENT",entity='cs21m003_cs21d406')
  wandb.agent(sweep_id, function=run_wandb, count=50)


if __name__ == '__main__':
  do_hyperparameter_search_using_wandb()  

Create sweep with ID: 732m5a9g
Sweep URL: https://wandb.ai/cs21m003_cs21d406/CS6910_ASSIGNMENT/sweeps/732m5a9g


wandb: Agent Starting Run: ppzi8pd4 with config:
wandb: 	act_func1: tanh
wandb: 	batch_size1: 16
wandb: 	epochs1: 10
wandb: 	hidden_layers1: 4
wandb: 	hidden_nodes1: 32
wandb: 	learning_rate1: 0.001
wandb: 	loss_func1: mse
wandb: 	optimizer1: rmsprop
wandb: 	reg1: 0
wandb: 	weight_init1: xavier
wandb: Currently logged in as: akashsainics21m003 (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


4431872/4422102 [==============================] - 0s 0us/step


Training_Accuracy_mse,▁▅▅▇▇▆▇▇██
Training_Loss_mse,█▄▄▂▂▃▂▂▁▁
Validation_Accuracy_mse,▁▅▆▇▇▇▇▇██
Validation_Loss_mse,█▄▄▂▂▂▂▂▁▁
epochs_mse,▁▂▃▃▄▅▆▆▇█
test_accuracy_mse,▁
Training_Accuracy_mse,0.87174
Training_Loss_mse,0.09289
Validation_Accuracy_mse,0.85083
Validation_Loss_mse,0.10753
epochs_mse,10


wandb: Agent Starting Run: q4w83lwi with config:
wandb: 	act_func1: tanh
wandb: 	batch_size1: 64
wandb: 	epochs1: 5
wandb: 	hidden_layers1: 4
wandb: 	hidden_nodes1: 64
wandb: 	learning_rate1: 0.001
wandb: 	loss_func1: mse
wandb: 	optimizer1: adam
wandb: 	reg1: 0.0005
wandb: 	weight_init1: xavier
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


...

Training_Accuracy_mse,▁▁▁▁▁▁▁▁▁▁
Training_Loss_mse,█▇▆▅▄▄▃▂▂▁
Validation_Accuracy_mse,▁▁▁▁▁▁▁▁▁▁
Validation_Loss_mse,█▇▆▅▄▄▃▂▂▁
epochs_mse,▁▂▃▃▄▅▆▆▇█
test_accuracy_mse,▁
Training_Accuracy_mse,0.1005
Training_Loss_mse,3.01977
Validation_Accuracy_mse,0.0955
Validation_Loss_mse,3.02477
epochs_mse,10


wandb: Agent Starting Run: 8vqmvifh with config:
wandb: 	act_func1: sigmoid
wandb: 	batch_size1: 16
wandb: 	epochs1: 10
wandb: 	hidden_layers1: 5
wandb: 	hidden_nodes1: 128
wandb: 	learning_rate1: 0.0001
wandb: 	loss_func1: mse
wandb: 	optimizer1: adam
wandb: 	reg1: 0.0005
wandb: 	weight_init1: random
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Training_Accuracy_mse,▁█▂█▃█▇█▃▇
Training_Loss_mse,█▄▂▁▁▁▁▁▁▁
Validation_Accuracy_mse,█▁▇▁▆▁▂▁▆▂
Validation_Loss_mse,█▄▂▁▁▁▁▁▁▁
epochs_mse,▁▂▃▃▄▅▆▆▇█
test_accuracy_mse,▁
Training_Accuracy_mse,0.10031
Training_Loss_mse,0.48326
Validation_Accuracy_mse,0.09717
Validation_Loss_mse,0.48329
epochs_mse,10


wandb: Agent Starting Run: 1vls9ses with config:
wandb: 	act_func1: relu
wandb: 	batch_size1: 32
wandb: 	epochs1: 5
wandb: 	hidden_layers1: 3
wandb: 	hidden_nodes1: 128
wandb: 	learning_rate1: 0.001
wandb: 	loss_func1: mse
wandb: 	optimizer1: nadam
wandb: 	reg1: 0.0005
wandb: 	weight_init1: random
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Training_Accuracy_mse,▁███▁
Training_Loss_mse,█▁▁▁▁
Validation_Accuracy_mse,█▁▁▁█
Validation_Loss_mse,█▁▁▁▁
epochs_mse,▁▃▅▆█
test_accuracy_mse,▁
Training_Accuracy_mse,0.09974
Training_Loss_mse,0.45
Validation_Accuracy_mse,0.10233
Validation_Loss_mse,0.45001
epochs_mse,5
